In [16]:
# encoding: UTF-8
import re
import os
import numpy as np
import scipy.io as sio
from Bio import SeqIO

In [2]:
# 把一个氨基酸序列转换为一个ONE HOT二维数组。氨基酸序列长度为N，数组是N-by-21，21表示 'GAVLIPFYWSTCMNQDEKRHX'这21个氨基酸缩写字符
def seq2OneHotArray(sequence):
    N = len(sequence)
    result = np.zeros(shape=(N,21),dtype=np.int16)
    alphabet = 'GAVLIPFYWSTCMNQDEKRHX'
    char_to_int = dict((c, i) for i, c in enumerate(alphabet))
    digseq = [char_to_int[char] for char in sequence]
    for i in range(N):
        result[i][digseq[i]] = 1
    return result        

In [4]:
def establishBenchmarkOneHot(fastaFile, saveFile):
    SeqOneHot = {}
    for seq_record in SeqIO.parse(fastaFile, 'fasta'):
        s = seq_record.seq
        r = seq2OneHotArray(s)
        SeqOneHot[seq_record.name] = r
   
    sio.savemat(saveFile,SeqOneHot)
    

In [14]:
# 滑窗。不足的补0
def splitWindowsWithZero(ws, matFile, sitesFile, saveFile):
    SeqOneHot = sio.loadmat(matFile)
    X = []
    Y = []
    k = 0
    t = 0
    with open(sitesFile, 'r') as pbsreader:
        for line in pbsreader:
            line = line.strip()
            if '>' in line:
                sid = line[1:]
                p = SeqOneHot[sid]
                seqlen = p.shape[0]
                col = np.size(p,1)
                for j in range(seqlen):
                    #create a array
                    d = np.zeros(shape=(ws*2+1,col), dtype=np.int16)
                    
                    if j < ws:
                        d[ws-j:2*ws+1] = p[0: ws+j+1]
                    elif j > seqlen - ws -1:
                        d[0:ws+seqlen-j] = p[j-ws:]
                    else:
                        d[::]=p[j-ws:j+ws+1]
                
                    X.insert(k,d)
                    Y.append([1,0])
                    k += 1
            else:
                sites = line.split()
                for s in sites:
                    idx = eval(s)
                    Y[t + idx -1] = [0,1]
                t=k  
    #save benchmark data set
    dataset={}
    dataset['data']=X
    dataset['target'] = Y
    sio.savemat(saveFile,dataset)


In [17]:
# 滑窗，不足的以头尾循环补
def splitWindows(ws, matFile, sitesFile, saveFile):
    SeqOneHot = sio.loadmat(matFile)
    X = []
    Y = []
    k = 0
    t = 0
    with open(sitesFile, 'r') as pbsreader:
        for line in pbsreader:
            line = line.strip()
            if '>' in line:
                sid = line[1:]
                p = SeqOneHot[sid]
                seqlen = p.shape[0]
                col = np.size(p,1)
                for j in range(seqlen):
                    #create a array
                    d = np.zeros(shape=(ws*2+1,col), dtype=np.int16)
                    '''
                    if j < ws:
                        d[ws-j:2*ws+1] = p[0: ws+j+1]
                    elif j > seqlen - ws -1:
                        d[0:ws+seqlen-j] = p[j-ws:]
                    else:
                        d[::]=p[j-ws:j+ws+1]
                    '''
                    if j < ws:
                        d[0:ws-j] = p[j-ws:]
                        d[ws-j:2*ws+1] = p[0: ws+j+1]
                    elif j > seqlen - ws -1:
                        d[0:ws] = p[j-ws:j]
                        d[ws:ws + seqlen -j] = p[j:]
                        d[ws+seqlen-j:] = p[0:ws-seqlen+j+1]
                    else:
                        d[::]=p[j-ws:j+ws+1]
                        
                    X.insert(k,d)
                    Y.append([1,0])
                    k += 1
            else:
                sites = line.split()
                for s in sites:
                    idx = eval(s)
                    Y[t + idx -1] = [0,1]
                t=k  
    #save benchmark data set
    dataset={}
    dataset['data']=X
    dataset['target'] = Y
    sio.savemat(saveFile,dataset)


In [20]:
#establishBenchmarkOneHot('data\\PDNA-224.fasta','data\\PDNA-224-seqs-OneHot.mat')
splitWindows(17, 'data\\PDNA-224-seqs-OneHot.mat', 'data\\PDNA-224-binding-sites.txt', 'data\\PDNA-224-OneHot-13.mat')